<a href="https://colab.research.google.com/github/eshnil2000/google-colab/blob/master/federated_learning_basic_pytorch_syft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#First, let's perform standard training
#Install required modules
#%%capture
import warnings
warnings.simplefilter('ignore')

!git clone https://github.com/eshnil2000/google-colab.git
%cd google-colab
!pip install pytorch 
import torch
from torch import nn
from torch import optim


In [0]:
# PySyft Toy Dataset 
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
model = nn.Linear(2,1)

#Training function
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [3]:
#Call the training function
train()


tensor(0.8230)
tensor(0.5356)
tensor(0.3514)
tensor(0.2316)
tensor(0.1534)
tensor(0.1021)
tensor(0.0684)
tensor(0.0461)
tensor(0.0312)
tensor(0.0213)
tensor(0.0147)
tensor(0.0102)
tensor(0.0071)
tensor(0.0050)
tensor(0.0036)
tensor(0.0026)
tensor(0.0018)
tensor(0.0013)
tensor(0.0010)
tensor(0.0007)


In [4]:
model.train()

Linear(in_features=2, out_features=1, bias=True)

In [5]:
#Inspect the trained weights
model.weight

Parameter containing:
tensor([[ 0.9788, -0.0073]], requires_grad=True)

In [0]:
#Now let's implement Federated training, where we distribute the data to multiple workers, send them the model 
#so they can independently run model on their data, and share back the learned weights, without sharing/sending back the data

#Install required modules
%%capture
import warnings
warnings.simplefilter('ignore')

!pip install syft
import syft as sy
hook = sy.TorchHook(torch)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [0]:
#The training function
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [12]:
#Call the training function
train()

tensor(0.9363, requires_grad=True)
tensor(0.9908, requires_grad=True)
tensor(0.8230, requires_grad=True)
tensor(0.3995, requires_grad=True)
tensor(0.4891, requires_grad=True)
tensor(0.2251, requires_grad=True)
tensor(0.2836, requires_grad=True)
tensor(0.1301, requires_grad=True)
tensor(0.1642, requires_grad=True)
tensor(0.0753, requires_grad=True)
tensor(0.0950, requires_grad=True)
tensor(0.0436, requires_grad=True)
tensor(0.0550, requires_grad=True)
tensor(0.0252, requires_grad=True)
tensor(0.0318, requires_grad=True)
tensor(0.0146, requires_grad=True)
tensor(0.0184, requires_grad=True)
tensor(0.0085, requires_grad=True)
tensor(0.0107, requires_grad=True)
tensor(0.0049, requires_grad=True)


In [13]:
#Inspect the trained weights
model.weight

Parameter containing:
tensor([[0.9430, 0.0467]], requires_grad=True)